**Counter**

* A counter stores a single number, the count
* You can add increment or decrement the counter, but you can't decrement its value past 0.

* A counter problem is an initial value for the counter, and a goal value
* The search is for a sequence of increment and decrement operations that move the counter value from the initial value to the goal value

* No, there is nothing difficult about the problem!


In [9]:
from searchClientInterface import WorldState

# The successor function needs to make a deep copy of itself
import copy

class CounterWorldState(WorldState):

    # Internal state is just the 'value' attribute
    def __init__(self, initial):
        self._value = initial
    
    # Convenience function to make counter objects print nicely
    # when they appear in search diagnostic messages
    def __str__(self):
        return "{" + str(self._value) + "}"
    
    #  These two methods are REQUIRED to make cycle checking work in the search
    #  Notice they depend on the object's internal state, so they must
    #  be customized to each new kind of WorldState
    
    def __eq__(self, other):
        if isinstance(other, CounterWorldState):
            return self._value == other._value
        else:
            return False

    def __hash__(self):
        return hash(str(self._value))
    
    # NB: every successor state must deep copy the old state!
    
    def successors(self):
        candidates = [self.increment(), self.decrement()]
        return [c for c in candidates if c] 
    
    # One next state generator for each possible action.
    # The return value is (newstate, action) or None if the 
    # action cannot be applied in the state
    
    def decrement(self):
        if self._value > 0:
            s = copy.deepcopy(self)
            s._value = s._value - 1
            return((s, "decrement"))
        else:
            return None
                   
    def increment(self):
        s = copy.deepcopy(self)
        s._value = s._value + 1
        return((s, "increment"))
    

In [10]:
from searchClientInterface import Problem

# This class describes a problem by implementing initial() and
# isGoal(state)

class CounterProblem(Problem):
    def __init__(self, initial_value, goal_value):
        self._state = CounterWorldState(initial_value)
        self._goal_value = goal_value
        
    def initial(self):
        return self._state
    
    def isGoal(self, state):
        return state._value == self._goal_value

In [11]:
from searchClientInterface import BFSEvaluator
from searchFramework import aStarSearch

problem1 = CounterProblem(3, 6)
print(aStarSearch(problem1, BFSEvaluator()))

(['increment', 'increment', 'increment'], (0.015625, 8, 2, 5))


In [17]:
# Try BFS on another problem;  add verbosity and a limit

print(aStarSearch(CounterProblem(7, 2), BFSEvaluator(),10, 20))

Visited 10 world is {6}
Skipped 3 Fringe is size 3
Evaluation is 3 with actions 3
(['decrement', 'decrement', 'decrement', 'decrement', 'decrement'], (0.0, 19, 8, 5))


In [18]:
# Try DFS on the first problem
from searchClientInterface import DFSEvaluator
print(aStarSearch(problem1, DFSEvaluator()))

(['increment', 'increment', 'increment'], (0.0, 4, 0, 4))


In [19]:
# DFS on the second problem (decrement)
print(aStarSearch(CounterProblem(7, 2), DFSEvaluator(), 250, 1000))

Visited 250 world is {256}
Skipped 0 Fringe is size 249
Evaluation is -249 with actions 249
Visited 500 world is {506}
Skipped 0 Fringe is size 499
Evaluation is -499 with actions 499
Visited 750 world is {756}
Skipped 0 Fringe is size 749
Evaluation is -749 with actions 749
Visited 1000 world is {1006}
Skipped 0 Fringe is size 999
Evaluation is -999 with actions 999
(None, (0.234375, 1001, 0, 1001))


In [22]:
# Here is real A* search with our heuristic evaluation being the distance
# between the current state and the goal value

from searchClientInterface import Evaluator

# Set up the problem
initial = 7
goal = 2
problem2 = CounterProblem(initial,goal)

# Define our A* evaluator
def coster(actions):
    return len(actions)
def estimator(state):
    return abs(state._value - goal)

# Run the search
evaluator = Evaluator(coster, estimator)
(solution, stats) = aStarSearch(problem2, evaluator, 250, 1000)
print(solution)
print(stats)
(solution, stats) = aStarSearch(problem2, BFSEvaluator(), 250, 1000)
print(solution)
print(stats)

['decrement', 'decrement', 'decrement', 'decrement', 'decrement']
(0.0, 6, 0, 6)
['decrement', 'decrement', 'decrement', 'decrement', 'decrement']
(0.0, 19, 8, 5)
